In [36]:
import numpy as np
import random
import cv2
from PIL import Image
from matplotlib import pyplot as plt


In [37]:
# table for adjusting the warmth of a photo
kelvin_table = {
    1000: (255,56,0),
    1500: (255,109,0),
    2000: (255,137,18),
    2500: (255,161,72),
    3000: (255,180,107),
    3500: (255,196,137),
    4000: (255,209,163),
    4500: (255,219,186),
    5000: (255,228,206),
    5500: (255,236,224),
    6000: (255,243,239),
    6500: (255,249,253),
    7000: (245,243,255),
    7500: (235,238,255),
    8000: (227,233,255),
    8500: (220,229,255),
    9000: (214,225,255),
    9500: (208,222,255),
    10000: (204,219,255)}

In [38]:
def salt_and_pepper(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

In [46]:
def dark_black_and_white(photo_name):
    img = Image.open(photo_name)
    threshold = 200
    fn = lambda x : 255 if x > threshold else 0
    r = img.convert('L').point(fn, mode='1')
    return r

In [40]:
def invert(image):
    image = ~image
    return image

In [41]:
def convert_temp(image, temp):
    r, g, b = kelvin_table[temp]
    matrix = ( r / 255.0, 0.0, 0.0, 0.0,
               0.0, g / 255.0, 0.0, 0.0,
               0.0, 0.0, b / 255.0, 0.0 )
    return image.convert('RGB', matrix)

In [42]:
def desaturate(image,scale,percent):
    # separate b,g,r
    b,g,r = cv2.split(image)
    b = b.astype(np.float32)
    g = g.astype(np.float32)
    r = r.astype(np.float32)

    # convert to cmyk
    c = 1 - r / scale
    m = 1 - g / scale
    y = 1 - b / scale
    k = cv2.min(cv2.min(c, m),y)
    c = scale * (c - k) / (1 - k)
    m = scale * (m - k) / (1 - k)
    y = scale * (y - k) / (1 - k)

    # desaturate neighbors of G which are C,Y
    c = cv2.multiply(c, percent)
    y = cv2.multiply(y, percent)

    # convert back to bgr
    r = scale * (1.0 - c / scale) * (1.0 - k)
    g = scale * (1.0 - m / scale) * (1.0 - k)
    b = scale * (1.0 - y / scale) * (1.0 - k)
    
    r = r.clip(0,255).astype(np.uint8)
    g = g.clip(0,255).astype(np.uint8)
    b = b.clip(0,255).astype(np.uint8)
    
    image_desaturated = cv2.merge([b,g,r])
    
    return image_desaturated

In [43]:
def disposable(image):
    rows, cols = image.shape[:2]
    # Create a Gaussian filter
    kernel_x = cv2.getGaussianKernel(cols,1500)
    kernel_y = cv2.getGaussianKernel(rows,1500)
    kernel = kernel_y * kernel_x.T
    filter = 2550 * kernel / np.linalg.norm(kernel)
    disposable_image = np.copy(image)
    # for each channel in the input image, we will apply the above filter
    for i in range(3):
        disposable_image[:,:,i] = disposable_image[:,:,i] * filter
    return disposable_image

In [44]:
def pop_art(original_image):
    # set colours (BGR)
    background_colour = [19,247,224]
    dots_colour = (247,19,217)

    # set the max dots (on the longest side of the image)
    max_dots = 120
    
    # extract dimensions
    original_image_height, original_image_width = original_image.shape

    # down size to number of dots
    if original_image_height == max(original_image_height,original_image_width):
        downsized_image = cv2.resize(original_image,(int(original_image_height*(max_dots/original_image_width)),max_dots))
    else:
        downsized_image = cv2.resize(original_image,(max_dots,int(original_image_height*(max_dots/original_image_width))))

    # extract dimensions of new image
    downsized_image_height, downsized_image_width = downsized_image.shape

    # set how big we want our final image to be
    multiplier = 100

    # set the size of our blank canvas
    blank_img_height = downsized_image_height * multiplier
    blank_img_width = downsized_image_width * multiplier

    # set the padding value so the dots start in frame (rather than being off the edge
    padding = int(multiplier/2)

    # create canvas containing just the background colour
    pop_art_image = np.full(((blank_img_height),(blank_img_width),3), background_colour,dtype=np.uint8)

    # run through each pixel and draw the circle on our blank canvas
    for y in range(0,downsized_image_height):
        for x in range(0,downsized_image_width):
            cv2.circle(pop_art_image,(((x*multiplier)+padding),((y*multiplier)+padding)), int((0.6 * multiplier) * ((255-downsized_image[y][x])/255)), dots_colour, -1)
    
    return pop_art_image


In [75]:
category_choice = int(input('Choose a type of photo to filter, press:\n(1) for Cars\n(2)'+
                        ' for Portraits\n(3) for Landscapes\nany other key to exit\n'))
if(category_choice == 1):
    category_directory = 'car_photos/'
    photo_name = str(input('enter a photo number from 1 to 25'))
    category_name = 'car'
    photo_directory = category_directory+category_name+photo_name+'.jpg'
    
elif(category_choice == 2):
    category_directory = 'portrait_photos/'
    category_name = 'portrait'
    photo_name = str(input('enter a photo number from 1 to 15'))
    photo_directory = category_directory+category_name+photo_name+'.jpg'

elif(category_choice == 3):
    category_directory = 'landscape_photos/'
    category_name = 'landscape'
    photo_name = str(input('enter a photo number from 1 to 15'))
    photo_directory = category_directory+category_name+photo_name+'.jpg'

filter_number = input('Choose a filter type, press:\n(1) for Salt & Pepper\n(2) for Black & White\n(3) for Inverting Filter'+
     '\n(4) for Temperature Change\n(5) for Desaturate\n(6) for Disposable\n(7) for Pop Art\n')    
if(filter_number == '1'):
    image = cv2.imread(photo_directory,0) # Only for grayscale image
    noise_img = salt_and_pepper(image,0.05)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_salt-pepper_filtered.jpg'
    cv2.imwrite(filtered_photo_name, noise_img)

if(filter_number == '2'):
    bw_image = dark_black_and_white(photo_directory)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_bw_filtered.jpg'
    bw_image.save(filtered_photo_name)

if(filter_number == '3'):
    image = cv2.imread(photo_directory)
    inverted_image = invert(image)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_inverted_filtered.jpg'
    cv2.imwrite(filtered_photo_name, inverted_image)

if(filter_number == '4'):
    # 10k = vibrant cold
    # 5000 = cold
    # 3000 = night blue
    # 1000 = dark electric blue, nightshade
    temparature = int(input('Input a temperature between 1k to 10k, for example 5k is cold, 3k is night blue and 10k is vibrant'))
    image = Image.open(photo_directory)
    cold_image = convert_temp(image,temparature)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_cold_filtered.jpg'
    cv2.imwrite(filtered_photo_name, np.array(cold_image))
    
if(filter_number == '5'):
    image = cv2.imread(photo_directory)
    scale = 255
    percent = float(input('Enter the percentage of desaturation, e.g 50 or 60 (or any other)'))/100
    image_desaturated = desaturate(image,scale,percent)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_desaturated.jpg'
    cv2.imwrite(filtered_photo_name, image_desaturated)
    
if(filter_number == '6'):
    image = cv2.imread(photo_directory)
    disposable_image = disposable(image)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_disposable.jpg'
    cv2.imwrite(filtered_photo_name, disposable_image)

if(filter_number == '7'):
    image = cv2.imread(photo_directory,0)
    pop_art_image = pop_art(image)
    filtered_photo_name = 'filtered_photos/'+category_name+'/'+category_name+photo_name+'_pop_filtered.jpg'
    cv2.imwrite(filtered_photo_name, pop_art_image)
    

Choose a type of photo to filter, press:
(1) for Cars
(2) for Portraits
(3) for Landscapes
any other key to exit
 3
enter a photo number from 1 to 15 1
Choose a filter type, press:
(1) for Salt & Pepper
(2) for Black & White
(3) for Inverting Filter
(4) for Temperature Change
(5) for Desaturate
(6) for Disposable
(7) for Pop Art
 1
